In [ ]:
import sys
sys.path.append('../')
from stim_surface_code.stability import StabilityPatch
from stim_surface_code import noise

# Basic usage

In [ ]:
s = StabilityPatch(6, 6, 6, 'Z')
s

In [ ]:
# 10x better noise vals than Google's device
noise_params = noise.GoogleNoiseParams.improve_noise_params(10)

print(noise_params.baseline_error_means)

In [ ]:
s.set_error_vals_normal(noise_params.baseline_error_means, noise_params.baseline_error_stdevs)

In [ ]:
s.count_logical_errors()

# Increasing d

In [ ]:
ds = [4,6,8]
for d in ds:
    s = StabilityPatch(d, d, d)
    s.set_error_vals_normal(noise_params.baseline_error_means, noise_params.baseline_error_stdevs)
    ler = s.count_logical_errors(10**7)[0]
    print(ler)